In [42]:
from __future__ import annotations

from abc import ABC, abstractmethod
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Dict

@dataclass
class Instruction(ABC):
    duration : int
    
    @abstractmethod
    def apply(self, x : int) -> int:
        raise NotImplementedError
    
    @classmethod
    def from_cmd(cls : Instruction, cmd : str, *args) -> Instruction:
        if cmd == 'noop':
            return Noop(0)
        elif cmd == 'addx':
            return Add(1, *args)
        
@dataclass
class Noop(Instruction):
    def apply(self, x : int) -> int:
        return x
    
@dataclass
class Add(Instruction):
    value : str
    def apply(self, x : int) -> int:
        #print(f'{x} + {self.value}')
        return x + int(self.value)

class Processor:
    def __init__(self, path):
        self.X : int = 1
        self.instructions : Dict[int, List[Instruction]] = defaultdict(list)
        self.result = 0
        self.path = path
        self.curr_instruction : Instruction = None
        
    def get_next_instruction(self, f):
        l = f.readline().rstrip('\r\n')
        if not l:
            return None
        
        parts = l.split(' ')
        cmd = parts[0]
        cmd_args = parts[1:]
        return Instruction.from_cmd(cmd, *cmd_args)
    
    def check_current_instruction(self):
        if not self.curr_instruction:
            return
        self.curr_instruction.duration -= 1
        if self.curr_instruction.duration < 0:
            self.X = self.curr_instruction.apply(self.X)
            self.curr_instruction = None

    def is_visible_pos(self, pos):
        return pos >= self.X - 1 and pos <= self.X + 1

    def execute(self):
        cycle = 0
        self.result = 0
        crt_line = ""
        with open(self.path, 'r') as f:
            while True:
                cycle += 1

                self.check_current_instruction()
                if not self.curr_instruction:
                    self.curr_instruction = self.get_next_instruction(f)

                crt_line += "#" if self.is_visible_pos((cycle-1)%40) else '.'
                if len(crt_line) == 40:
                    print(crt_line)
                    crt_line = ""

                if cycle == 20 or (cycle - 20) % 40 == 0:
                    strength = cycle*self.X
                    self.result += strength
                    
                if not self.curr_instruction:
                    break
                

p = Processor('../day10.txt')
p.execute()
print(f'X:{p.result}')

###..####.###...##....##.####.#....#..#.
#..#....#.#..#.#..#....#.#....#....#.#..
###....#..#..#.#..#....#.###..#....##...
#..#..#...###..####....#.#....#....#.#..
#..#.#....#....#..#.#..#.#....#....#.#..
###..####.#....#..#..##..####.####.#..#.
X:11220
